没有rag的版本
直接通过与LLM的交互

In [2]:
from openai import OpenAI

# 初始化客户端
client = OpenAI(api_key="sk-7e37bea1ecc249968d28f386eb60b09e", base_url="https://api.deepseek.com")

# 定义专家配置
experts = [
    {"name": "神经科学专家", "prefix": "假如你是一个神经科学专家"},
    {"name": "深度学习专家", "prefix": "假如你是一个深度学习专家"},
    {"name": "外科医生", "prefix": "假如你是一个外科医生"}
]

# 初始化对话历史（每个专家独立的对话历史）
histories = {expert["name"]: [] for expert in experts}

# 获取初始问题
user_question = input("请输入您的问题: ")
print("\n" + "="*50)

# 主循环（10轮对话）
for round in range(3):
    print(f"\n{' 第' + str(round+1) + '轮对话 ':=^50}")
    
    # 阶段1: 专家回答
    for expert in experts:
        expert_name = expert["name"]
        
        # 构建当前消息
        if round == 0:  # 第一轮添加专家前缀
            current_message = f"{expert['prefix']}，{user_question}"
        else:  # 后续轮次直接使用用户反馈
            current_message = histories[expert_name][-1]["content"]
        
        # 更新对话历史
        histories[expert_name].append({"role": "user", "content": current_message})
        
        # 调用API获取专家回答
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=histories[expert_name]
        )
        
        # 获取并保存回答
        expert_reply = response.choices[0].message.content
        histories[expert_name].append({"role": "assistant", "content": expert_reply})
        
        # 打印专家回答
        print(f"\n【{expert_name}的回答】")
        print(expert_reply)
        print("-"*50)
    
    # 最后一轮不收集反馈
    if round == 4:
        break
        
    # 阶段2: 收集用户反馈
    print("\n" + "="*50)
    print(f"\n{' 请提供反馈 ':=^50}")
    for expert in experts:
        expert_name = expert["name"]
        feedback = input(f"请对{expert_name}的回答给出反馈: ")
        histories[expert_name].append({"role": "user", "content": feedback})

    st = input("input q to exit the system, input other things to continue:")
    if st == "q":
        break
    
print("\n" + "="*50)



===================== 第1轮对话 ======================

【神经科学专家的回答】
作为神经科学专家，设计基于人工视网膜的视觉修复系统需要整合多学科知识（神经工程、材料科学、计算机视觉等）。以下是分步骤的实现框架：

---

### **1. 系统核心目标**
- **适用人群**：视网膜色素变性（RP）、年龄相关性黄斑变性（AMD）等光感受器退化患者。
- **功能替代**：通过人工装置替代受损的光感受器，将光信号转化为电信号，刺激残存视网膜神经元（如双极细胞或神经节细胞）。

---

### **2. 关键组件与技术实现**

#### **（1）图像采集与处理**
- **微型摄像头**：植入眼镜或眼内，实时捕捉场景。
- **预处理算法**：
  - 动态范围压缩（适应明暗变化）
  - 边缘增强（突出关键轮廓）
  - 降噪（减少信息冗余）
  - *示例*：仿生视网膜的“中心-外周抑制”处理（模拟自然视网膜的神经节细胞感受野）。

#### **（2）人工视网膜植入体**
- **电极阵列类型**：
  - **表面型**（如Argus II）：刺激神经节细胞，需高电流但易植入。
  - **穿透型**（如犹他电极阵列）：直接刺激双极细胞，分辨率更高但手术风险大。
- **材料选择**：
  - 生物兼容性材料（如聚二甲硅氧烷PDMS、氮化钛电极）。
  - 柔性基底（减少组织损伤）。

#### **（3）信号编码策略**
- **脉冲编码模式**：
  - 亮度→脉冲频率调制（如Phosphene感知）。
  - 空间信息→多电极时空激活模式（需匹配视网膜拓扑图）。
- **个性化校准**：患者通过反馈训练系统优化信号-感知映射。

#### **（4）无线供能与数据传输**
- **射频耦合**（如Argus II外部线圈供电）。
- **近红外光供能**（实验阶段，可实现全植入式设计）。

---

### **3. 临床挑战与解决方案**
- **分辨率限制**：
  - 现状：Argus II仅60电极，提供粗略光点（Phosphene）。
  - 改进方向：高密度阵列（如波士顿视网膜植入体的256电极）+ 动态电流聚焦技术。
- **生物相容性**：
  - 涂层技术（如PEDOT:PSS

In [9]:
from langchain_community.document_loaders import (
    DirectoryLoader,
    TextLoader,
    PyPDFLoader,
    Docx2txtLoader,
)

In [11]:
# 指定加载文档的目录
LOAD_PATH = "surgery"

def load_documents(source_dir: str):
    """
    加载指定目录下的所有文档
    支持格式：.txt, .pdf, .docx, .md
    """

    # 分别加载不同格式，txt，md 格式
    text_loader = DirectoryLoader(
        path=source_dir,  # 指定读取文件的父目录
        glob=["**/*.txt", "**/*.md"],  # 指定读取文件的格式
        show_progress=True,  # 显示加载进度
        use_multithreading=True,  # 使用多线程
        silent_errors=True,  # 错误时不抛出异常，直接忽略该文件
        loader_cls=TextLoader,  # 指定加载器
        loader_kwargs={"autodetect_encoding": True},  # 自动检测文件编码
    )
    # pdf 格式
    pdf_loader = DirectoryLoader(
        path=source_dir,
        glob="**/*.pdf",
        show_progress=True,
        use_multithreading=True,
        silent_errors=True,
        loader_cls=PyPDFLoader,
    )
    # docx 格式
    docx_loader = DirectoryLoader(
        path=source_dir,
        glob="**/*.docx",
        show_progress=True,
        use_multithreading=True,
        silent_errors=True,
        loader_cls=Docx2txtLoader,
        loader_kwargs={"autodetect_encoding": True},
    )
    # 合并文档列表
    docs = []
    docs.extend(text_loader.load())
    docs.extend(pdf_loader.load())
    docs.extend(docx_loader.load())
    print(f"成功加载 {len(docs)} 份文档")
    return docs

documents = load_documents(LOAD_PATH)

# 测试是否成功加载文档
for doc in documents[:2]:  # 打印前两篇摘要
    print(f"文件路径: {doc.metadata['source']}")
    print(f"内容预览: {doc.page_content[:150]}...\n")

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:11<00:00, 11.43s/it]

KeyboardInterrupt: 

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=800, chunk_overlap=150):
    """
    使用递归字符分割器处理文本
    参数说明：
    - chunk_size：每个文本块的最大字符数，推荐 500-1000
    - chunk_overlap：相邻块之间的重叠字符数（保持上下文连贯），推荐 100-200
    """
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", "。", "!", "?", "？", "！", "；", ";"],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True,  # 保留原始文档中的位置信息
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"原始文档数：{len(documents)}")
    print(f"分割后文本块数：{len(split_docs)}")

    # 查看分割效果示例
    print("\n示例文本块：")
    print(split_docs[0].page_content[:300] + "...")
    print(f"元数据：{split_docs[0].metadata}")

    return split_docs

# 执行分割
split_docs = split_documents(documents)

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
import time
from langchain_community.embeddings import DashScopeEmbeddings

# 指定持久化向量数据库的存储路径
VECTOR_DIR = "surgery_vector"

def create_vector_store(split_docs, persist_dir=VECTOR_DIR):
    """
    创建持久化向量数据库
    :param split_docs: 经过分割的文档列表
    :param persist_dir: 向量数据库存储路径（建议使用WSL原生路径）
    """

    # 初始化本地嵌入模型
    embeddings = DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key="sk-b90be4e81ec04f1bada3c70f4368bebb")
    try:
        start_time = time.time()

        # 创建带进度显示的向量数据库
        db = Chroma.from_documents(
            documents=split_docs,
            embedding=embeddings,
            persist_directory=persist_dir,  # 持久化存储路径
        )

        print(f"\n向量化完成！耗时 {time.time()-start_time:.2f} 秒")
        print(f"数据库存储路径：{persist_dir}")
        print(f"总文档块数：{db._collection.count()}")

        return db
    except Exception as e:
        print(f"向量化失败：{str(e)}")
        return None


# 执行向量化（使用之前分割好的split_docs）
# vector_db = create_vector_store(split_docs)

In [5]:
from langchain_community.embeddings import DashScopeEmbeddings
embeddings = DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key="sk-b90be4e81ec04f1bada3c70f4368bebb")

如果已经完成向量化，从这开始运行就可以

In [6]:
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import readline
from langchain_openai import ChatOpenAI
# 模型名称
MODEL_NAME = "deepseek-chat"

# 构建检索链流程
def build_nr_chain(VECTOR_DIR , memory):
    # 1. 初始化向量数据库
    vector_store = Chroma(
        persist_directory=VECTOR_DIR,
        embedding_function=DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key="sk-b90be4e81ec04f1bada3c70f4368bebb"),
    )
    openai_api_key = "sk-7e37bea1ecc249968d28f386eb60b09e"  #deepseek key
    openai_api_base = "https://api.deepseek.com/v1"

    #2. 初始化LLM
    llm = ChatOpenAI(
        openai_api_key=openai_api_key,
        openai_api_base=openai_api_base,
        model = "deepseek-chat", #deepseek-reasoner
        temperature=0.7,
        stop=['Observation:', 'Observation:\n']
    )

    # 3. 初始化检索器，并设置检索参数
    retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "fetch_k": 20,
            "lambda_mult": 0.5,
            "score_threshold": 0.4,
        },
    )

    # 4. 设置提示词模板
    system_template = """
        假设你是一个神经科学专家。
        接下来讲给出一些从相关文档查询的知识，以及一个问题，请你进行思考。
        上下文：{context}
        """
    prompt = ChatPromptTemplate(
        [
            ("system", system_template),
            ("human", "{question}"),
        ]
    )
    # 构建 LangChain 检索链
    return (
        {
            "context": retriever,
            "question": RunnablePassthrough(),
            "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
        }
        | prompt
        | llm
        | StrOutputParser()
    )

# 构建检索链流程
def build_dl_chain(VECTOR_DIR , memory):
    # 1. 初始化向量数据库
    vector_store = Chroma(
        persist_directory=VECTOR_DIR,
        embedding_function=DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key="sk-b90be4e81ec04f1bada3c70f4368bebb"),
    )
    openai_api_key = "sk-7e37bea1ecc249968d28f386eb60b09e"  #deepseek key
    openai_api_base = "https://api.deepseek.com/v1"

    #2. 初始化LLM
    llm = ChatOpenAI(
        openai_api_key=openai_api_key,
        openai_api_base=openai_api_base,
        model = "deepseek-chat", #deepseek-reasoner
        temperature=0.7,
        stop=['Observation:', 'Observation:\n']
    )


    # 3. 初始化检索器，并设置检索参数
    retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "fetch_k": 20,
            "lambda_mult": 0.5,
            "score_threshold": 0.4,
        },
    )

    # 4. 设置提示词模板
    system_template = """
        假设你是一个深度学习专家。
        接下来讲给出一些从相关文档查询的知识，以及一个问题，请你进行思考。
        上下文：{context}
        """
    prompt = ChatPromptTemplate(
        [
            ("system", system_template),
            ("human", "{question}"),
        ]
    )
    # 构建 LangChain 检索链
    return (
        {
            "context": retriever,
            "question": RunnablePassthrough(),
            "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
        }
        | prompt
        | llm
        | StrOutputParser()
    )

# 构建检索链流程
def build_su_chain(VECTOR_DIR , memory):
    # 1. 初始化向量数据库
    vector_store = Chroma(
        persist_directory=VECTOR_DIR,
        embedding_function=DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key="sk-b90be4e81ec04f1bada3c70f4368bebb"),
    )
    openai_api_key = "sk-7e37bea1ecc249968d28f386eb60b09e"  #deepseek key
    openai_api_base = "https://api.deepseek.com/v1"

    #2. 初始化LLM
    llm = ChatOpenAI(
        openai_api_key=openai_api_key,
        openai_api_base=openai_api_base,
        model = "deepseek-chat", #deepseek-reasoner
        temperature=0.7,
        stop=['Observation:', 'Observation:\n']
    )

    # 3. 初始化检索器，并设置检索参数
    retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "fetch_k": 20,
            "lambda_mult": 0.5,
            "score_threshold": 0.4,
        },
    )

    # 4. 设置提示词模板
    system_template = """
        假设你是一个外科医生。
        接下来讲给出一些从相关文档查询的知识，以及一个问题，请你进行思考。
        如果相关知识没有你想要的，请独立思考。
        上下文：{context}
        """
    prompt = ChatPromptTemplate(
        [
            ("system", system_template),
            ("human", "{question}"),
        ]
    )
    # 构建 LangChain 检索链
    return (
        {
            "context": retriever,
            "question": RunnablePassthrough(),
            "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
        }
        | prompt
        | llm
        | StrOutputParser()
    )

In [7]:
from langchain.memory import ConversationBufferMemory
memory1 = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
memory2 = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
memory3 = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
chain1 = build_nr_chain("neuroscience_vector", memory1)
chain2 = build_dl_chain("deeplearning_vector", memory2)
chain3 = build_su_chain("surgery_vector", memory3)

# 交互界面
print("系统就绪，输入问题开始对话（输入 'exit' 退出）")
print(f"\n{' 第1轮对话 ':=^50}")
response1 = ""
response2 = ""
response3 = ""
query = input("\n初始问题：").strip()
query1 = query
query2 = query
query3 = query
# 回答采用流式输出，invoke 将问题传入到 Runnables 管道中
for chunk in chain1.invoke(query1):
    response1 += chunk
print("=" * 100)
print("\n神经科学专家的回答是：")
print("=" * 100)
print(response1)

for chunk in chain2.invoke(query2):
    response2 += chunk
print("=" * 100)
print("\n深度学习专家的回答是：")
print("=" * 100)
print(response2)

for chunk in chain3.invoke(query3):
    response3 += chunk
print("=" * 100)
print("\n外科专家的回答是：")
print("=" * 100)
print(response3)

for i in range(2):
    split_string = lambda str: (
        str.split("</think>", 1)[1] if "</think>" in str else str
    )
    # 将当前对话的问题和回答，保存到记忆缓冲区中
    memory1.save_context({"inputs": query1}, {"outputs": split_string(response1)})
    memory2.save_context({"inputs": query2}, {"outputs": split_string(response2)})
    memory3.save_context({"inputs": query3}, {"outputs": split_string(response3)})
    print("\n\n")
    query1 = input("请输入对于神经科学专家的评价，输入exit退出")
    query2 = input("请输入对于深度学习专家的评价，输入exit退出")
    query3 = input("请输入对于外科专家的评价，输入exit退出")
    if query1 == "exit" or query2 == "exit" or query3 == "exit":
        break
    print(f"\n{' 第' + str(i+2) + '轮对话 ':=^50}")
    response1 = ""
    response2 = ""
    response3 = ""
    for chunk in chain1.invoke(query1):
        response1 += chunk
    print("=" * 100)
    print("\n神经科学专家的回答是：")
    print("=" * 100)
    print(response1)

    for chunk in chain2.invoke(query2):
        response2 += chunk
    print("=" * 100)
    print("\n深度学习专家的回答是：")
    print("=" * 100)
    print(response2)

    for chunk in chain3.invoke(query3):
        response3 += chunk
    print("=" * 100)
    print("\n外科专家的回答是：")
    print("=" * 100)
    print(response3)
   


/var/folders/0x/vwp7kdd14n73yn71fv6wbyhw0000gn/T/ipykernel_19756/2298225683.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory1 = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


系统就绪，输入问题开始对话（输入 'exit' 退出）

===================== 第1轮对话 ======================


KeyboardInterrupt: 

初始化知识库系统...
系统就绪，输入问题开始对话（输入 'exit' 退出）
回答：根据文档内容，深度学习是通过学习多层次的转换来进行多层次的表示学习的一类机器学习方法。它不仅取代了传统机器学习的浅层模型，还取代了劳动密集型的特征工程。深度学习已经彻底改变了模式识别，引入了一系列技术，包括计算机视觉、自然语言处理、自动语音识别等。



==== 请继续对话（输入 'exit' 退出）====


In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import readline

VECTOR_DIR = "myvector"
MODEL_NAME = "deepseek-r1:7b"

# 初始化会话记忆缓冲区，用于存储对话历史，保存在内存中
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

def build_qa_chain():

    vector_store = Chroma(
        persist_directory=VECTOR_DIR,
        embedding_function=DashScopeEmbeddings(model="text-embedding-v1", dashscope_api_key="sk-b90be4e81ec04f1bada3c70f4368bebb"),
    )

    openai_api_key = "sk-7e37bea1ecc249968d28f386eb60b09e"  #deepseek key
    openai_api_base = "https://api.deepseek.com/v1"

    llm = ChatOpenAI(
        openai_api_key=openai_api_key,
        openai_api_base=openai_api_base,
        model = "deepseek-chat",
        # model = "qvq-max",
        # model = "qwq-32b",
        temperature=0.7,
        stop=['Observation:', 'Observation:\n']
    )

    retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "fetch_k": 20,
            "lambda_mult": 0.5,
            "score_threshold": 0.4,
        },
    )

    system_template = """
        您是一名一个设计用于査询文档来回答问题的代理。
        您可以使用文档检索工具，并基于检索内容来回答问题。
        您可能不查询文档就知道答案，但是您仍然应该查询文档来获得答案。
        如果您从文档中找不到任何信息用于回答问题，则只需返回“抱歉，这个问题我还不知道。”作为答案。
        如果有人提问等关于您的名字的问题，您就回答：“我是小天才助手”作为答案。
        上下文：{context}
        """
    prompt = ChatPromptTemplate(
        [
            ("system", system_template),
            MessagesPlaceholder("chat_history"),  # 将历史对话插入到模板中
            ("human", "{question}"),
        ]
    )

    return (
        {
            "question": RunnablePassthrough(),
            "context": retriever,
            "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
        }
        | prompt
        | llm
        | StrOutputParser()
    )

def console_qa():
    print("初始化知识库系统...")
    chain = build_qa_chain()
    print("系统就绪，输入问题开始对话（输入 'exit' 退出）")
    while True:
        try:
            query = input("\n问题：").strip()
            if not query or query.lower() in ("exit", "quit"):
                break

            print("回答：", end="", flush=True)
            response = ""
            
            for chunk in chain.invoke(query):
                response += chunk
            print(response)
            # 截取 </think> 后面的字符串
            split_string = lambda str: (
                str.split("</think>", 1)[1] if "</think>" in str else str
            )
            # 将当前对话的问题和回答，保存到记忆缓冲区中
            memory.save_context({"inputs": query}, {"outputs": split_string(response)})

            print("\n\n")
            print("==== 请继续对话（输入 'exit' 退出）====")

        except KeyboardInterrupt:
            break


if __name__ == "__main__":
    console_qa()
    print("对话结束")

初始化知识库系统...
系统就绪，输入问题开始对话（输入 'exit' 退出）
回答：根据文档内容，深度学习是机器学习的一个分支，它通过以下特点进行定义：

1. **多层次表示学习**：深度学习通过学习多层次的转换（神经网络层）来自动提取数据的多层次抽象表示，属于表示学习的一种。

2. **取代传统方法**：它不仅替代了传统机器学习的浅层模型（如线性回归、SVM等），还消除了人工设计特征的繁琐过程（特征工程），实现了端到端的学习。

3. **驱动因素**：近年来的突破主要得益于：
   - 大规模数据（来自廉价传感器和互联网应用）
   - 算力提升（如GPU的广泛应用）

4. **应用领域**：深度学习彻底改变了计算机视觉、自然语言处理、语音识别等模式识别任务，并成为许多先进应用（如医疗诊断、自动驾驶）的核心技术。

简单来说，深度学习是通过多层神经网络自动学习数据复杂表示的机器学习方法，能够直接从原始数据中提取高级特征并优化整体系统性能。



==== 请继续对话（输入 'exit' 退出）====
回答：以下是基于PyTorch实现Transformer核心模块的Python代码示例（参考《动手学深度学习》内容）：

```python
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(drop